In [35]:
import os
import re
import logging
from langchain.agents import AgentExecutor,  initialize_agent # Tool,
from langchain_core.tools import Tool
from langchain.docstore.document import Document
from langchain.agents.agent_types import AgentType
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain_experimental.utilities import PythonREPL
from langchain_openai import ChatOpenAI
import clickhouse_connect

from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
#from langchain_chroma import Chroma

In [9]:
# Настройка логирования
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler('logs.log')
fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
logger.addHandler(fh)
logger.info("START_LOGGING")

In [10]:
# Конфигурация безопасности
os.environ['AGENT_MODE'] = 'STRICT'  # Запрет управления агентом

In [11]:
# подключение к БД параметры подключения - в команде запуска контейнера
def connect_to_base(host, port, user, password):
    '''
    Подключение к БД
    '''
    try:
        client = clickhouse_connect.get_client(host= host, port= port, username= user, password= password)
        logger.debug("Connection to the database is successful")
        return client
    except:
        logger.error(f"Error loading to connect: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Error loading the model: {str(e)}")

#client = connect_to_base(os.environ.get(CH_HOST), os.environ.get(CH_PORT), os.environ.get(CH_USER), os.environ.get(CH_PASSWORD))

In [12]:
# Инструмент 1: Безопасный ClickHouse запрос
def safe_clickhouse_query(query: str) -> str:
    """Выполняет SQL-запросы только для чтения с валидацией"""
    # Защита от инъекций
    forbidden_keywords = ['insert', 'update', 'delete', 'drop', 'alter', 'create', 'grant']
    if any(re.search(rf'\b{kw}\b', query.lower()) for kw in forbidden_keywords):
        return "Ошибка: Запрещенная операция"
    
    # Только SELECT/SHOW/DESCRIBE
    if not re.match(r'^\s*(select|show|describe|with|explain)', query, re.IGNORECASE):
        return "Ошибка: Разрешены только запросы чтения"
    
    try:
        result = client.query(query)
        return str(result.result_rows)
    except Exception as e:
        return f"Ошибка запроса: {str(e)}"

In [4]:
loader = TextLoader("db_schema_docs.md", encoding="utf-8")

In [14]:
loader

In [5]:
documents = loader.load()

In [13]:
documents[0]

Document(metadata={'source': 'db_schema_docs.md'}, page_content="# Таблица: also_viewed\nОписание: товары, которые смотрели клиенты при покупке\n## **shop_id**: ID магазина (uint32)\n## **client_id**: ID клиента (uint64)\n## **items**: ID Товаров (Float32)\n## **date**: дата (datetime64)\n\n# Таблица: bonus_histories\nОписание: история начисления бонусов\n##  **code**  :    код                   (uint64)       \n##  **shop_id**  :  ID магазина           (uint32)        \n##  **client_id** :  ID клиента           (uint64)        \n##  **status**   :    статус                (string)       \n##  **total_balance** :  итоговый баланс бонусов     (float64)       \n##  **used** :          использовано бонусов   (float64)       \n##   **burned**  :  истек срок годности бонусов     (float64)       \n##  **cancelled** :  отменено банусов            (float64)       \n##  **inactive**  :     неактивно бонусов       (float64)       \n##  **rewarded** :     начисленно бонусов     (float64)       \n

In [46]:
file_md = open('db_schema_docs.md', encoding='utf-8').read()

In [47]:
file_md

"# Таблица: also_viewed  Описание: товары, которые смотрели клиенты при покупке\n## **shop_id**: ID магазина (uint32)\n## **client_id**: ID клиента (uint64)\n## **items**: ID Товаров (Float32)\n## **date**: дата (datetime64)\n\n# Таблица: bonus_histories\nОписание: история начисления бонусов\n##  **code**  :    код                   (uint64)       \n##  **shop_id**  :  ID магазина           (uint32)        \n##  **client_id** :  ID клиента           (uint64)        \n##  **status**   :    статус                (string)       \n##  **total_balance** :  итоговый баланс бонусов     (float64)       \n##  **used** :          использовано бонусов   (float64)       \n##   **burned**  :  истек срок годности бонусов     (float64)       \n##  **cancelled** :  отменено банусов            (float64)       \n##  **inactive**  :     неактивно бонусов       (float64)       \n##  **rewarded** :     начисленно бонусов     (float64)       \n##  **purchase_date** :  дата покупки          (datetime64)\n## 

In [48]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
]

In [49]:
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

In [50]:
def get_chunks(splitter, text):
    chunks = []
    for chunk in splitter.split_text(text):
        # Если chunk - объект Document, используем его page_content
        if hasattr(chunk, 'page_content'):
            page_content = chunk.page_content
            metadata = getattr(chunk, 'metadata', {}).copy()  # Копируем существующие метаданные, если они есть
            metadata.update({"meta": "data"})  # Обновляем их
            chunks.append(Document(page_content=page_content, metadata=metadata))
        else:
            # Если chunk - строка (на всякий случай)
            chunks.append(Document(page_content=chunk, metadata={"meta": "data"}))
    return chunks

In [51]:
chunks = get_chunks(text_splitter, file_md)

In [59]:
chunks[0]

Document(metadata={'Header 1': 'Таблица: bonus_histories', 'meta': 'data'}, page_content='Описание: история начисления бонусов')

In [17]:
FAISS.from_documents(docs, embeddings)

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://router.huggingface.co/hf-inference/models/mixedbread-ai/mxbai-embed-large-v1/pipeline/feature-extraction

In [15]:
# Инструмент 2: RAG для схемы базы данных
def setup_rag_agent():
    """Инициализация векторной базы знаний о схеме"""
    loader = TextLoader("db_schema_docs.md", encoding="utf-8")
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru")
    return FAISS.from_documents(docs, embeddings)

vector_db = setup_rag_agent()

def schema_retriever(query: str) -> str:
    """Поиск информации о структуре БД"""
    docs = vector_db.similarity_search(query, k=3)
    return "\n\n".join([d.page_content for d in docs])

ReadTimeout: (ReadTimeoutError("HTTPSConnectionPool(host='cdn-lfs.hf.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 6bb759a8-6317-4f11-9117-888f9c1096f0)')

In [12]:
# Инструмент 3: Python для сложных вычислений
python_repl = PythonREPL()

In [13]:
# Инициализация инструментов
tools = [
    Tool(
        name="ClickHouse_Query",
        func=safe_clickhouse_query,
        description=(
            "Выполнение SQL-запросов к ClickHouse. Только SELECT/SHOW/DESCRIBE. "
            "Вход: валидный SQL-запрос. Выход: результат таблицы."
        )
    ),
    Tool(
        name="Database_Schema",
        func=schema_retriever,
        description=(
            "Поиск информации о структуре базы данных. "
            "Используй для уточнения имен таблиц и столбцов. "
            "Вход: естественный язык."
        )
    ),
    Tool(
        name="Python_REPL",
        func=python_repl.run,
        description=(
            "Выполнение Python кода для сложных вычислений. "
            "Используй только когда невозможно решить через SQL. "
            "Вход: валидный Python код."
        )
    )
]

In [14]:
# Системный промпт с ограничениями
system_prompt = f"""
Ты senior data analyst. Правила:
1. Только чтение данных (SELECT/SHOW/DESCRIBE)
2. Запрещены DDL/DML операции
3. Для работы с БД используй инструменты
4. Все ответы на русском
5. Формат вывода: таблица Markdown

Доступные базы:
{schema_retriever("Общее описание схемы")}
"""

NameError: name 'vector_db' is not defined

In [15]:
# Инициализация агента
agent = initialize_agent(
    tools=tools,
    llm=ChatOpenAI(api_key="sk-f8b7345eb09247468b17932c9884b04c",
                   base_url="https://api.deepseek.com/v1",
                   model="deepseek-chat",
                   temperature=0),
    agent=AgentType.OPENAI_FUNCTIONS,
    agent_kwargs={
        'system_message': system_prompt,
        'extra_prompt_messages': [
            "Важно: Все SQL-запросы проверяй через Database_Schema!",
            "Ограничение: Максимум 5 строк в Python_REPL"
        ]
    },
    verbose=True,
    handle_parsing_errors=True
)

NameError: name 'system_prompt' is not defined

In [16]:
# Пример использования
if __name__ == "__main__":
    query = "Выгрузи средний чек по месяцам для магазина с id=123 за 2024 год"
    result = agent.invoke({"input": query})
    print(f"\nРезультат:\n{result['output']}")

NameError: name 'agent' is not defined